In [ ]:
from data_processing import Dataset
from noise import NoiseScheduler
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torchvision import transforms
import torch


In [ ]:
data_dir = "/cephfs/dice/users/ek19824/l1trigger/diffusion/datasets"       # set to directory where data is stored

dataset = Dataset(1_000, (120, 72), signal_file=f"{data_dir}/CaloImages_signal.root", pile_up_file=f"{data_dir}/CaloImages_bkg.root", save=False)

In [ ]:
dataset() #once this is cached, you don't have to re-load

In [ ]:
new_dim=(64,64)

In [ ]:
dataset.preprocess(16, new_dim)     # ensure these have the same values as when the model was trained

In [ ]:
preprocess = transforms.Compose(
        [
            transforms.ToTensor()
        ]
)

In [ ]:
clean_frames = preprocess(dataset.signal).float().permute(1, 2, 0).unsqueeze(1) #pytorch symantics
pile_up = preprocess(dataset.pile_up).float().permute(1, 2, 0).unsqueeze(1)

In [ ]:
### Select one image to perform inference on
i = 10      # image 
noise_scheduler = NoiseScheduler('pile-up')
timestep = torch.LongTensor([40])   # each unit of timestep represents an addtional 5 pile-up events (max 40 = 200PU)
random_seed = 42
noisy_image, noise = noise_scheduler.add_noise(clean_frame=clean_frames[i], noise_sample=pile_up, timestep=timestep, random_seed=random_seed, n_events = 1000)

In [ ]:
plt.imshow(noisy_image.reshape(new_dim))
plt.colorbar()
plt.title("Noised Image")

In [ ]:
from models import Model, UNetLite, UNetLite_hls
modtype = 'UNet_lite'

if modtype == 'UNet2d':
    model = Model('UNet', new_dim)
elif modtype == 'UNet_lite':
    model = UNetLite_hls()

print(model)

In [ ]:
if modtype == 'UNet2d':
    model = model.__getitem__()

In [ ]:
if modtype == 'UNet2d':
    trained_model_path = '/hdfs/user/ys20884/hackathon/trained_models/trained_diffusor.pt'
elif modtype == 'UNet_lite':
    trained_model_path = 'trained_models_lite/model_epoch_9.pt'

print(trained_model_path)

In [ ]:
checkpoint = torch.load(trained_model_path)

In [ ]:
model.load_state_dict(checkpoint)

In [ ]:
if modtype == 'UNet2d':
    noise_pred = model(noisy_image.unsqueeze(1), timestep, return_dict=False)[0]
elif modtype == 'UNet_lite':
    noise_pred = model(noisy_image.unsqueeze(1), timestep)       #, return_dict=False)[0]

ims = plt.imshow(noise_pred.detach().numpy().reshape(new_dim))
plt.colorbar(ims)
plt.title("Noise Prediction")

In [ ]:
de_noised = noisy_image[0] - (noise_pred.detach().numpy().reshape(new_dim))

In [ ]:

im = plt.imshow(de_noised.reshape(new_dim))
plt.colorbar(im)
plt.title("Denoised Image")

In [ ]:
im2 = plt.imshow(clean_frames[i].squeeze())
plt.colorbar(im2)
plt.title("Clean Image")

In [ ]:
err = clean_frames[i].squeeze()-de_noised.reshape(new_dim)
print(f"MSE: {np.mean((err.numpy())**2)}")
